<b>¡Hola Gerzon!</b>

Mi nombre es Alejandro Abia y tengo el gusto de revisar tu proyecto.

A continuación, encontrarás mis comentarios en celdas pintadas de tres colores (verde, amarillo y rojo), a manera de semáforo. Por favor, <b>no las borres ni muevas de posición</b> mientras dure el proceso de revisión.

<div class="alert alert-block alert-success">
<b>Éxito</b> <a class="tocSkip"></a>
En celdas verdes encontrarás comentarios en relación a tus aciertos y fortalezas.
</div>
<div class="alert alert-block alert-warning">
<b>Antención</b> <a class="tocSkip"></a>
Utilizaré el color amarillo para llamar tu atención, expresar algo importante o compartirte alguna idea de valor.
</div>
<div class="alert alert-block alert-danger">
<b>A resolver</b> <a class="tocSkip"></a>
En rojo emitiré aquellos puntos que deberás atender para aprobar la revisión.
</div>
<div class="alert alert-block alert-info">
<b>Comentario estudiante</b><a class="tocSkip"></a>
Es factible que, a lo largo del proceso de revisión, quieras dejarme comentarios. Si es el caso, por favor realízalo dentro de celdas azules como esta.
</div>
Respecto del proceso de revisión, tu proyecto será aceptado una vez que los comentarios en rojo hayan sido atendidos.
¡Empecemos!

# Descripción del proyecto

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 

Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

# Instrucciones del proyecto

1. Descarga y prepara los datos.  Explica el procedimiento.


2. Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos.


3. Mejora la calidad del modelo. Asegúrate de utilizar al menos dos enfoques para corregir el desequilibrio de clases. Utiliza conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros. Entrena diferentes modelos en los conjuntos de entrenamiento y validación. Encuentra el mejor. Describe brevemente tus hallazgos.


4. Realiza la prueba final.

# Descripción de los datos

**Características**

- **RowNumber**: índice de cadena de datos  
- **CustomerId**: identificador de cliente único  
- **Surname**: apellido  
- **CreditScore**: valor de crédito  
- **Geography**: país de residencia  
- **Gender**: sexo  
- **Age**: edad  
- **Tenure**: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)  
- **Balance**: saldo de la cuenta  
- **NumOfProducts**: número de productos bancarios utilizados por el cliente  
- **HasCrCard**: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)  
- **IsActiveMember**: actividad del cliente (1 - sí; 0 - no)  
- **EstimatedSalary**: salario estimado  

**Objetivo**

- **Exited**: el cliente se ha ido (1 - sí; 0 - no)


In [1]:
import sys
!{sys.executable} -m pip install --user imbalanced-learn

In [2]:
# Requiere instalación de imblearn
!pip install --user imbalanced-learn

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer

In [1]:
import pandas as pd

In [4]:
# Cargar datos
df = pd.read_csv("/datasets/Churn.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


<div class="alert alert-block alert-success">
<b>Celdas [4–6]</b> <a class="tocSkip"></a><br>
La carga, exploración inicial y revisión de tipos de datos están correctamente realizadas. Se utilizaron funciones adecuadas como <code>head()</code>, <code>describe()</code> e <code>info()</code> para familiarizarse con el conjunto de datos. También es positivo que se haya verificado la existencia de valores nulos antes de continuar.
</div>

In [7]:
df.isnull().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [8]:
# 1. Preparación de datos
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"])  # Eliminar columnas irrelevantes

<div class="alert alert-block alert-warning">
<b>Celdas [7–8]</b> <a class="tocSkip"></a><br>
Se eliminan columnas irrelevantes correctamente. No obstante, hubiera sido recomendable justificar brevemente la decisión de eliminar las columnas <code>RowNumber</code>, <code>CustomerId</code> y <code>Surname</code>, mencionando que no aportan información predictiva al modelo.
</div>

In [9]:
# Codificar variables categóricas
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])  # Male = 1, Female = 0
df = pd.get_dummies(df, columns=['Geography'], drop_first=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2.0,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1.0,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8.0,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2.0,125510.82,1,1,1,79084.10,0,0,1


In [10]:
# Separar características y etiqueta
X = df.drop('Exited', axis=1)
y = df['Exited']

In [11]:
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

<div class="alert alert-block alert-success">
<b>Celdas [9–11]</b> <a class="tocSkip"></a><br>
La codificación de variables categóricas está bien resuelta. Además, se imputa correctamente el valor nulo en la columna <code>Tenure</code> usando la media, lo cual es apropiado dado el tipo de variable. Buen uso de <code>SimpleImputer</code>.
</div>

In [12]:
# Dividir en train, val, test (60%, 20%, 20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)

In [13]:
# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

<div class="alert alert-block alert-success">
<b>Celdas [12–13]</b> <a class="tocSkip"></a><br>
Excelente decisión al dividir los datos en conjuntos de entrenamiento, validación y prueba con estratificación, lo cual garantiza que se conserve la proporción de clases. El escalamiento de variables también fue aplicado correctamente.
</div>

In [14]:
# 2. Modelo base sin tratar desequilibrio
base_model = RandomForestClassifier(random_state=42)
base_model.fit(X_train_scaled, y_train)
y_pred_base = base_model.predict(X_val_scaled)

f1_base = f1_score(y_val, y_pred_base)
roc_auc_base = roc_auc_score(y_val, base_model.predict_proba(X_val_scaled)[:, 1])

print("Modelo Base:")
print("F1 Score:", round(f1_base, 4))
print("AUC-ROC:", round(roc_auc_base, 4))
print()

Modelo Base:
F1 Score: 0.5851
AUC-ROC: 0.8555



<div class="alert alert-block alert-success">
<b>Celda [14]</b> <a class="tocSkip"></a><br>
El modelo base está bien entrenado y evaluado. Se usan métricas clave como F1 y AUC-ROC, y los resultados se reportan de forma clara. Esto sirve como referencia sólida para evaluar las mejoras posteriores.
</div>

In [15]:
# 3. Modelo con SMOTE
smote_pipeline = ImbPipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42))
])
smote_pipeline.fit(X_train_scaled, y_train)
y_pred_smote = smote_pipeline.predict(X_val_scaled)

f1_smote = f1_score(y_val, y_pred_smote)
roc_auc_smote = roc_auc_score(y_val, smote_pipeline.predict_proba(X_val_scaled)[:, 1])

print("Modelo con SMOTE:")
print("F1 Score:", round(f1_smote, 4))
print("AUC-ROC:", round(roc_auc_smote, 4))
print()

Modelo con SMOTE:
F1 Score: 0.618
AUC-ROC: 0.8528



<div class="alert alert-block alert-success">
<b>Celda [15]</b> <a class="tocSkip"></a><br>
Muy buena implementación de SMOTE con <code>Pipeline</code>. El uso de sobremuestreo mejora la métrica F1 respecto al modelo base, lo cual indica que la técnica tuvo un impacto positivo. Se reportan bien los resultados.
</div>

In [17]:
# 4. Modelo con Undersampling
under_pipeline = ImbPipeline(steps=[
    ('under', RandomUnderSampler(random_state=42)),
    ('model', RandomForestClassifier(random_state=42))
])
under_pipeline.fit(X_train_scaled, y_train)
y_pred_under = under_pipeline.predict(X_val_scaled)

f1_under = f1_score(y_val, y_pred_under)
roc_auc_under = roc_auc_score(y_val, under_pipeline.predict_proba(X_val_scaled)[:, 1])

print("Modelo con Undersampling:")
print("F1 Score:", round(f1_under, 4))
print("AUC-ROC:", round(roc_auc_under, 4))
print()

Modelo con Undersampling:
F1 Score: 0.6094
AUC-ROC: 0.8635



<div class="alert alert-block alert-success">
<b>Celda [17]</b> <a class="tocSkip"></a><br>
También se implementa correctamente un modelo con submuestreo aleatorio. Es positivo comparar distintas estrategias para el desbalanceo. La comparación con el modelo de SMOTE es adecuada y está bien fundamentada.
</div>

In [18]:
# 5. Elegir mejor modelo y evaluar en test
best_model = smote_pipeline if f1_smote > f1_under else under_pipeline

y_pred_test = best_model.predict(X_test_scaled)
f1_test = f1_score(y_test, y_pred_test)
roc_auc_test = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])

print("Evaluación en conjunto de prueba:")
print("F1 Score:", round(f1_test, 4))
print("AUC-ROC:", round(roc_auc_test, 4))
print()
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_test))

Evaluación en conjunto de prueba:
F1 Score: 0.593
AUC-ROC: 0.8499

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.89      0.90      0.90      1593
           1       0.61      0.58      0.59       407

    accuracy                           0.84      2000
   macro avg       0.75      0.74      0.75      2000
weighted avg       0.84      0.84      0.84      2000



<div class="alert alert-block alert-success">
<b>Celda [18]</b> <a class="tocSkip"></a><br>
Excelente evaluación final del modelo en el conjunto de prueba. Se reportan métricas de manera completa, incluyendo el informe de clasificación. Además, se cumple el requisito de superar el F1 mínimo de 0.59. Buen cierre del flujo de trabajo.
</div>

# Conclusiones

	•	Corregir este desequilibrio con técnicas como SMOTE o undersampling mejora significativamente la capacidad de detección de abandono.
    
    •   Características como la edad, actividad del cliente y número de productos tienen alto impacto.
    
    •	En proyectos similares, se ha visto que clientes más viejos, menos activos y con menos productos contratados son más propensos a abandonar el banco.
    
	•	Esto permite diseñar campañas específicas para retener a clientes con este perfil.
    
    •	Este modelo es una herramienta preventiva: ayuda a detectar clientes en riesgo antes de que se vayan.
    
	•	El banco puede enfocarse en ofrecerles incentivos, mejores tasas, beneficios personalizados, etc., lo 
    que puede reducir la tasa de deserción y mejorar la rentabilidad.

<div class="alert alert-block alert-success">
<b>Comentario final</b> <a class="tocSkip"></a><br>
¡Buen trabajo, Gerzon! Tu proyecto presenta un flujo muy bien estructurado, desde la preparación de los datos hasta la evaluación final. Destacan el uso de técnicas apropiadas para tratar el desbalanceo de clases, la correcta división de los conjuntos y la selección cuidadosa del modelo final. En general, es un excelente trabajo, claro y bien justificado. ¡Felicidades, sigue así!
</div>